# Data Preparation

In [1]:
%matplotlib inline

import pandas
from sqlalchemy import create_engine

## Download the data

In [2]:
url = ("http://data.insideairbnb.com/france/ile-de-france/paris"
       "/2019-09-16/data/listings.csv.gz")
%time db = pandas.read_csv(url)

<string>:2: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 9.02 s, sys: 4.61 s, total: 13.6 s
Wall time: 2min 22s


In [3]:
tokeep = [
    "id", 
    "neighbourhood_cleansed",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "bed_type"
    ]
review = [i for i in db.columns if "review_scores" in i]

## Build numeric price

In [4]:
cleaner = lambda p: float(p.replace(",", "").strip("$"))
db["Price"] = db["price"].apply(cleaner)

## Keep relevant variables

In [5]:
final = db[tokeep + review + ["Price"]].dropna()

## Write out

* As `.csv`

In [6]:
! rm -f /paris_abb.csv.zip
final[tokeep + ["Price"]].to_csv("paris_abb.csv.zip",
                                 index=False)

* As `.sqlite`

In [7]:
! rm -f /paris_abb_mini.db
engine = create_engine("sqlite:///paris_abb_mini.db")

In [8]:
final[tokeep + ["Price"]].to_sql("db", engine, index=False)

Index on `neighbourhood_cleansed`

In [9]:
qry = """
CREATE INDEX idx_neighbourhood_cleansed
ON db (neighbourhood_cleansed);
"""
conn = engine.connect()
conn.execute(qry)

In [10]:
out = conn.execute("select * from SQLite_master")

for table in out.fetchall():
    print(table[4])

CREATE TABLE db (
	id BIGINT, 
	neighbourhood_cleansed TEXT, 
	property_type TEXT, 
	room_type TEXT, 
	accommodates BIGINT, 
	bathrooms FLOAT, 
	bedrooms FLOAT, 
	beds FLOAT, 
	bed_type TEXT, 
	"Price" FLOAT
)
CREATE INDEX idx_neighbourhood_cleansed
ON db (neighbourhood_cleansed)


In [11]:
conn.close()
engine.dispose()

* Full table dump on sqlite (optional, final file was eventually removed from the pack as 270MB)

In [12]:
! rm -f paris_abb.db
engine = create_engine("sqlite:///paris_abb.db")

In [13]:
db.rename(columns={"Price": "price_val"})\
  .to_sql("db", engine, index=False)

Index on `neighbourhood_cleansed`

In [14]:
qry = """
CREATE INDEX idx_neighbourhood_cleansed
ON db (neighbourhood_cleansed);
"""
conn = engine.connect()
conn.execute(qry)

In [15]:
out = conn.execute("select * from SQLite_master")

for table in out.fetchall():
    print(table[4])

CREATE TABLE db (
	id BIGINT, 
	listing_url TEXT, 
	scrape_id BIGINT, 
	last_scraped TEXT, 
	name TEXT, 
	summary TEXT, 
	space TEXT, 
	description TEXT, 
	experiences_offered TEXT, 
	neighborhood_overview TEXT, 
	notes TEXT, 
	transit TEXT, 
	access TEXT, 
	interaction TEXT, 
	house_rules TEXT, 
	thumbnail_url FLOAT, 
	medium_url FLOAT, 
	picture_url TEXT, 
	xl_picture_url FLOAT, 
	host_id BIGINT, 
	host_url TEXT, 
	host_name TEXT, 
	host_since TEXT, 
	host_location TEXT, 
	host_about TEXT, 
	host_response_time TEXT, 
	host_response_rate TEXT, 
	host_acceptance_rate FLOAT, 
	host_is_superhost TEXT, 
	host_thumbnail_url TEXT, 
	host_picture_url TEXT, 
	host_neighbourhood TEXT, 
	host_listings_count FLOAT, 
	host_total_listings_count FLOAT, 
	host_verifications TEXT, 
	host_has_profile_pic TEXT, 
	host_identity_verified TEXT, 
	street TEXT, 
	neighbourhood TEXT, 
	neighbourhood_cleansed TEXT, 
	neighbourhood_group_cleansed FLOAT, 
	city TEXT, 
	state TEXT, 
	zipcode TEXT, 
	market TEXT,

In [16]:
conn.close()
engine.dispose()

* Review scores

In [17]:
! rm -f paris_abb_review.csv.zip
final[["id"] + review].to_csv("paris_abb_review.csv.zip",
                              index=False)